In [ ]:
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats
import sys, os
sys.path.append(os.path.abspath("../data_creation/plots_and_analysis"))
import config
# adjust paths in /data_creation/plots_and_analysis/config.py


In [ ]:
# get accuracy metrics
all_metrics_df = pd.read_parquet(f"{config.path_to_evaluation}/results.parquet")
all_metrics_df["div_step"] = all_metrics_df["div_step"].astype(str)
all_metrics_df.rename(columns={"div_step": "div", "pot target size": "p"}, inplace=True)
all_metrics_df


In [ ]:
# append target features
mean_target_features = pd.read_parquet(f"{config.path_to_evaluation}/target_features_num.parquet")
all_metrics_df = all_metrics_df.merge(mean_target_features,on=["p", "div", "run"], how="outer")
all_metrics_df


In [ ]:
all_metrics_df.columns

In [ ]:
# load source features
mean_source_features = pd.read_parquet(f"{config.path_to_evaluation}/source_features_num.parquet")
mean_source_features

In [ ]:
# merge with source features
# Merge the dataframes on 'div' and 'run' keys
merged_df = all_metrics_df.merge(
    mean_source_features, 
    on=['div', 'run'], 
    how='left', 
    suffixes=('', '_source')
)

# Fill missing values in rows where p='source'
mask = merged_df['p'] == 'source'

feature_columns = ['abs_energy', 'intermittency', 
                   'mean', 'median', 'kurtosis', 'skewness', 'standard_deviation', 
                   'agg_autocorrelation_max', 'erraticness', 'agg_linear_trend_slope']

for col in feature_columns:
    if f'{col}_source' in merged_df.columns:
        merged_df.loc[mask, col] = merged_df.loc[mask, col].fillna(merged_df.loc[mask, f'{col}_source'])

source_cols = [col for col in merged_df.columns if col.endswith('_source')]
merged_df = merged_df.drop(columns=source_cols)

all_metrics_df = merged_df
all_metrics_df.rename(columns={"agg_autocorrelation_max":"autocorr", "agg_linear_trend_slope":"trend", "standard_deviation":"sd"}, inplace=True)
all_metrics_df.to_parquet(f"{config.path_to_evaluation}/results_metrics_features.parquet")
all_metrics_df

To avoid collinearity in the linear regression, calculate the Pearson correlation matrix.

In [ ]:
features = ['abs_energy', 'intermittency', 'mean', 'median', 'kurtosis', 
                     'skewness', 'sd', 'autocorr', 
                     'erraticness', 'trend']

div = "aggregated" #choose between: "0", "5", "10", "aggregated"

if div == "aggregated":
    complete_data = all_metrics_df[features].dropna() 
else:
    complete_data = all_metrics_df.loc[all_metrics_df["div"] == div][features].dropna()
    
correlation_matrix = complete_data.corr()

plt.rcParams.update({
    'font.size': 12,
    'font.weight': 'bold',
    'axes.labelweight': 'bold',
    'axes.titleweight': 'bold',
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'xtick.major.width': 2,
    'ytick.major.width': 2,
    'axes.linewidth': 2
})

plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, fmt='.2f', 
            annot_kws={'weight': 'bold', 'size': 10}) 
plt.tight_layout()  
plt.savefig(f"{config.path_to_evaluation}/feature_correlations_div{div}.pdf",  bbox_inches='tight')
plt.show()

Choose feature sets with little collinearity and run a multiple linear regression.

In [ ]:
all_metrics_df["RMSSE"]

In [ ]:
div = "aggregated" # choose between: "0", "5", "10", "aggregated"

# chosen feature set
features = ['autocorr', 'mean', 'skewness', 'trend']
#['abs_energy','intermittency',  'trend',  'erraticness']
#[ 'autocorr', 'mean', 'skewness', 'trend'] 
#['intermittency',  'mean',  'kurtosis', 'trend'] 

# Define targets
targets = ['MASE', 'RMSSE', 'sMAPE']

all_columns = features + targets
if div == "aggregated":
    complete_data = all_metrics_df[all_columns].dropna()
else:
    complete_data = all_metrics_df.loc[all_metrics_df["div"] == div][all_columns].dropna()

X = complete_data[features]
X_with_const = sm.add_constant(X)

# Create separate dictionaries to store results for each target
results_dict = {}

for target in targets:
    y = complete_data[target]
    
    # Fit the model
    model = sm.OLS(y, X_with_const).fit()
    
    # Store coefficients, p-values, and R2 for this target
    results_dict[target] = {
        'coefficients': model.params,
        'pvalues': model.pvalues,
        'r2': model.rsquared
    }

# Create the final DataFrame with desired structure
feature_names = ['const'] + features

# Create columns with R2 in parentheses next to target names
columns = [('Feature', '')]
for target in targets:
    r2_formatted = f"{target} (R2={results_dict[target]['r2']:.2f})"
    columns.extend([
        (r2_formatted, 'Coefficient'),
        (r2_formatted, 'P-value')
    ])

multi_columns = pd.MultiIndex.from_tuples(columns)

data_rows = []
for feature in feature_names:
    row_data = [feature] 
    for target in targets:
        row_data.extend([
            round(results_dict[target]['coefficients'][feature], 2),
            round(results_dict[target]['pvalues'][feature], 4)
        ])
    data_rows.append(row_data)


final_df = pd.DataFrame(data_rows, columns=multi_columns)

print("Results Summary:")
print(final_df)

final_df.to_csv(f"{config.path_to_evaluation}/ols_{features}_all_metrics_div{div}.csv")

Plot the erraticness against the RMSSE and MASE and the intermittency against the sMAPE. Fit a linear line and calculate the slope and p-value.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(20, 6))
df = all_metrics_df

unique_pot_target_sizes = df["p"].unique()
colors = plt.cm.tab20(np.linspace(0, 1, len(unique_pot_target_sizes)))

def plot_linear_fit(ax, x, y, metric):
    if len(x) > 1:
        # Linear regression
        linear_coeffs = np.polyfit(x, y, deg=1)
        linear_eq = np.poly1d(linear_coeffs)
        x_fit = np.linspace(min(x), max(x), 100)
        y_fit = linear_eq(x_fit)
        
        ax.plot(x_fit, y_fit, linestyle="--", alpha=0.9, linewidth=5, color='black')
        
        scatter_plots = []
        for i, size in enumerate(unique_pot_target_sizes):
            mask = df["p"] == size
            scatter_plot = ax.scatter(x[mask], y[mask], c=[colors[i]], s=250, edgecolors="w", alpha=0.9, linewidth=2)
            scatter_plots.append(scatter_plot)

        slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)

        if metric == 'sMAPE':
            ax.annotate(f"Slope: {slope:.3f}\np-value: {p_value:.3e}",
                        xy=(0.05, 0.95), xycoords='axes fraction', fontsize=18,
                        ha='left', va='top',
                        bbox=dict(boxstyle="round,pad=0.3", edgecolor='black', facecolor='lightgray'))
        elif metric in ['RMSSE', 'MASE']:
            ax.annotate(f"Slope: {slope:.3f}\np-value: {p_value:.3e}",
                        xy=(0.95, 0.95), xycoords='axes fraction', fontsize=18,
                        ha='right', va='top',
                        bbox=dict(boxstyle="round,pad=0.3", edgecolor='black', facecolor='lightgray'))
        
        return scatter_plots[0]  

# Intermittency vs. sMAPE
scatter1 = plot_linear_fit(axes[0], df["intermittency"], df["sMAPE"], "sMAPE")
axes[0].set_xlabel("Intermittency", fontsize=24)
axes[0].set_ylabel("sMAPE", fontsize=24)
axes[0].set_title("Intermittency vs. sMAPE", fontsize=24)
axes[0].grid(True)

# Erraticness vs. RMSSE
scatter2 = plot_linear_fit(axes[1], df["erraticness"], df["RMSSE"], "RMSSE")
axes[1].set_xlabel("Erraticness", fontsize=24)
axes[1].set_ylabel("RMSSE", fontsize=24)
axes[1].set_title("Erraticness vs. RMSSE", fontsize=24)
axes[1].grid(True)

# Erraticness vs. MASE
scatter3 = plot_linear_fit(axes[2], df["erraticness"], df["MASE"], "MASE")
axes[2].set_xlabel("Erraticness", fontsize=24)
axes[2].set_ylabel("MASE", fontsize=24)
axes[2].set_title("Erraticness vs. MASE", fontsize=24)
axes[2].grid(True)


legend_elements = []
for i, size in enumerate(unique_pot_target_sizes):
    legend_elements.append(plt.Line2D([0], [0], marker='o', color='w', 
                                    markerfacecolor=colors[i], markersize=12, 
                                    markeredgecolor='white', markeredgewidth=0.5, 
                                    label=size))

fig.legend(title="Potential Target Size", handles=legend_elements, loc='upper center', bbox_to_anchor=(0.5, 0.02), 
          ncol=len(unique_pot_target_sizes), fontsize=18, frameon=True, title_fontsize=18)

plt.tight_layout()
plt.subplots_adjust(bottom=0.2) 
plt.savefig(f"{config.path_to_evaluation}/single_ols_erraticness_intermittency.pdf",  bbox_inches='tight')
plt.show()